In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
from fractions import Fraction
import warnings
warnings.filterwarnings('ignore')


In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'BTU'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Window Blinds"""
buckets = """Massage & Manipulation Tools"""

regex_pattern_inch=r'(?i)(cord length measures\s?\d+.?\d*.?\")|(cord measures\s?\d+.?\d*.?\")|()'
regex_pattern_foot=r"(?i)((cord length measures\s?\d+.?\d*.?ft))|((cord length measures\s?\d+.?\d*.?\'))|((cord measures\s?\d+.?\d*.?ft))|((cord measures\s?\d+.?\d*.?\'))|()"

value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [4]:
df = query_from_file(file_name='query/uncurated.sql', params=params)
df

product_id external_id  \
0    988783774    45548536   
1   1070151724    45548499   
2    817167835    44676643   
3    817165406    43765393   
4   1085625706    69676984   
..         ...         ...   
60   817347897    67028726   
61   817673495    66304937   
62   817673496    66304944   
63  1165794438    62970747   
64   817269973    62970815   

                                                                                product_name  \
0                         Northwest Freestanding Mahogany Electric Fireplace Heater in Black   
1                                  Northwest Curved Glass Electric Fireplace Heater in Black   
2                                         Comfort Zone® Mini Ceramic Fireplace Heater in Red   
3                                                           Vornado® Small Room Tower Heater   
4                         Dr. Infrared Heater™ 240-volt Hardwired Shop Garage Heater in Grey   
..                                                                                       ...   
60                                            Teleshop Insta Heater Wall Outlet Space Heater   
61                                                           Lasko® Bladeless Ceramic Heater   
62                                                      Lasko® Ultra Ceramic Heater in Black   
63  Southern Enterprises Rosedale Corner Convertible Faux Stone Electric Fireplace in Sienna   
64             Southern Enterprises Redden Corner Convertible Electric Fireplace in Espresso   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                                           

In [68]:
df.to_csv("uncurated_eleven_unmapped_delete.csv")

In [67]:
df[df['external_id'].astype(str)=='69581630']

product_id external_id  \
48   988580683    69581630   

                                                   product_name  \
48  Le Creuset® Signature 1.75 qt. Covered Saucepan in Licorice   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        long_desc  \
48  Delivering quick, even heating, outstanding durability, plus an eye-catching look on any stovetop, the 1.75 qt. Le Creuset Signature Covered Saucepan makes preparing sauces easier than ever thanks to its smoothly curved interior and rounded base. Le Creuset Signature 1.75 qt. Covered Saucepan offers superb cooking performance, exceptional durability, and an eye-catching design to let you easily and beautifully make a wide array of sauces Enameled cast iron construction ensures rapid and even heat distribution plus outstanding heat retention Sand-colored enamel interior makes it easy to monitor food as it cooks while also helping prevent burning and sticking Vibrantly colored plus chip- and stain-resistant enameled exterior looks great on any cooktop Suitable for use on all stove types/heat sources, including induction Oven safe up to 500º F Snug-fitting enameled cast iron lid with ergonomic, cool-touch composite knob handle helps keep moisture and heat from escaping Comfortable and sturdy extended helper handle and a contoured main handle provide added control when transferring cookware from stove/oven to table Enameled cast iron Dishwasher safe Measures 13.75\" L x 6.5\" diameter x 5.5\" H 1.75 qt. capacity Limited lifetime manufacturer's warranty Imported   

    customer_id attributes    buckets values  
48            5       None  Saucepans   None

In [ ]:
error

In [5]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [6]:
def clean_data(data):
    data['matches']=data['matches'].str.get(0)
#     data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())

In [7]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_inch, x))
inch=df[df['matches'].astype(str)!='[]']
clean_data(inch) #might throw an error and have to drop nane
inch['matches']=inch['matches'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'\\",'',str(measure))).replace("'", '').apply(lambda quote: quote.replace('"', ''))
inch

product_id external_id  \
14   988683304    67178070   
40   817391428    69003943   
82   817375528    68398590   
91   988476164    69592377   

                                                    product_name  \
14                        truMedic® InstaShiatsu Pillow Massager   
40  Human Touch® Reflex SWING Pro Foot, Calf, and Thigh Massager   
82                  Brookstone® Max 2 Percussion Massager in Red   
91                           Sharper Image® BodyScan Seat Topper   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
14                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     Use the truMedic InstaShiatsu+ Pillow Massager while relaxing on a couch, chair or bed and feel the tension and strain leave your neck, back and shoulders. It lets you enjoy a relaxing Shiatsu massage anytime, in the privacy of your home. Helps relieve tension in neck, back and shoulders Heat setting adds additional comfort and relief 15-minute run time On/off switch Measures 6.5\" D x 5.5\" H x 13.5\" W Cord measures 76\" L 6-month manufacturer's warranty Imported Model IS-1000   
40  The Reflex SWING Pro Foot, Calf and Thigh Massager by Human Touch delivers the ultimate spa-like treatment in the comfort of your home. Massages feet, calves and thighs by emulating techniques used by professional massage therapists. Innovative Foot, Calf and Thigh Massager bridges state-of-the-art CirQlation™ and Figure-Eight Technologies with the ancient healing art of massage, emulating the same techniques used by massage professionals 2 speed settings 2 zones Massages calves in an upward, rolling or wave-like motion Figure-Eight movement caresses sore calves and rejuvenates tired feet to elevate relaxation while forcing blood away from the feet and toward the core of the body to improve circulation Massages all the way up to the thighs Incorporates reflexology that benefits and improves general health, relieving stress and pain in many other parts of the body Patented Warm-Air Technology and underfoot vibration, and you have a massage experience unlike any other One-touch control Measures 19.5\" L x 15\" W x 17.5\" H Cord measures 78\" L 1-year limited manufacturer's warranty Imported Model 200REFLEXSP001   
82                                                                                                            

# Foot

In [8]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_foot, x))
foot=df[df['matches'].astype(str)!='[]']
clean_data(foot) #might throw an error and have to drop nane
foot['matches']=foot['matches'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\\|\'",'',str(measure))).replace("'", '').apply(lambda quote: quote.replace('"', ''))
foot['matches']=foot['matches'].astype(float).apply(lambda x: float(x*12)).astype(int)
foot

product_id external_id  \
0     817476713    62598620   
15    988650641    65696040   
16    817567391    65695968   
17    817567386    65695913   
23    817381182    68715922   
28    817567389    65695951   
29    817186269    62090162   
34    817182980    61322714   
36    988481024    69604528   
43   1027808045    69639437   
47    817180304    60664969   
49    817607713    68398552   
51    817567387    65696026   
52    817306484    64997124   
57    817318619    65696019   
61    817607710    68398620   
62    817567388    65695944   
66    988480832    69604349   
69   1170775633    66857426   
74    817182979    61322707   
77    988476912    69595006   
78    817183712    61444508   
80    988476820    69594917   
83   1139460437    64997117   
87    817678107    68447724   
89    817318621    65695975   
94    988476887    69594997   
95    988476819    69594919   
104   817473407    61444478   
107   817607712    68398637   
109   817678113    68447779   

                                                                             product_name  \
0    HoMedics Cordless SHIATSUTALK™ Voice-Controlled Neck and Shoulder Massager with Heat   
15                           HoMedics Therapist Select 2.0 Foot & Calf Massager with Heat   
16                       HoMedics® Cordless True Touch Neck & Shoulder Massager with Heat   
17                                      HoMedics® 3D True-Touch Massage Cushion With Heat   
23                        HoMedics® Shiatsu Air Max Deep Kneading Foot Massager with Heat   
28                                          HoMedics® Air Compression Back-Stretching Mat   
29                                  HoMedics® Shower Bliss Foot Spa with Heat Boost Power   
34                                                      Wagan Heated Shiatsu Massage Belt   
36          HoMedics® 2-in-1 Shiatsu Massaging Seat Topper with Removeable Massage Pillow   
43                                             Westinghouse Multifunctional Foot Massager   
47        Aurora Health and Beauty Neck and Back Shiatsu Massager with Heat in Tan/Orange   
49                          Brookstone® 2-in-1 Tapping & Shiatsu Neck & Shoulder Massager   
51                                      HoMedics® Shiatsu Air Pro Foot Massager with Heat   
52            Aurora Health and Beauty® Compression Shiatsu Heated Foot Massager in White   
57                              HoMedics® Cordless Percussion Massager with Heat in White   
61                                                        Shiatsu Foot Massager with Heat   
62                             HoMedics® Shiatsu Recline Massaging Chaise Lounger in Grey   
66                           HoMedics® Gentle Touch Gel Shiatsu Massage Cushion with Heat   
69                                         Prospera® TENS Socks Electronic Pulse Massager   
74                                                   Wagan Heated Shiatsu Massage Cushion   
77                                       HoMedics® Gentle Touch Gel Foot Massager in Grey   
78                                                    Prospera Neck and Shoulder Massager   
80                                      HoMedics® Body Flex Air Compression Mat with Heat   
83                 Aurora Health and Beauty® Shiatsu Massage Heated Seat Cushion in Black   
87                                            Brookstone® C7 Shiatsu Massging Seat Topper   
89                                             HoMedics® Shiatsu Massage Pillow with Heat   
94                   HoMedics® Pro Therapy Vibration Neck Massager with Soothing in Black   
95          HoMedics® 2-in-1 Shiatsu Massaging Seat Topper with Removeable Massage Pillow   
104                                                   Prospera Personal Massager in White   
107                                                      Brookstone® Foot & Calf Massager   
109                                             Brookstone® Foot & Calf Massager in Black   

                                   

In [9]:
inch['matches'].explode().value_counts()

16.54    1
76       1
12       1
78       1
Name: matches, dtype: int64

In [10]:
foot['matches'].explode().value_counts()

72     26
60      2
864     2
120     1
Name: matches, dtype: int64

In [11]:
matches=pd.concat([inch,foot])
matches

product_id external_id  \
14    988683304    67178070   
40    817391428    69003943   
82    817375528    68398590   
91    988476164    69592377   
0     817476713    62598620   
15    988650641    65696040   
16    817567391    65695968   
17    817567386    65695913   
23    817381182    68715922   
28    817567389    65695951   
29    817186269    62090162   
34    817182980    61322714   
36    988481024    69604528   
43   1027808045    69639437   
47    817180304    60664969   
49    817607713    68398552   
51    817567387    65696026   
52    817306484    64997124   
57    817318619    65696019   
61    817607710    68398620   
62    817567388    65695944   
66    988480832    69604349   
69   1170775633    66857426   
74    817182979    61322707   
77    988476912    69595006   
78    817183712    61444508   
80    988476820    69594917   
83   1139460437    64997117   
87    817678107    68447724   
89    817318621    65695975   
94    988476887    69594997   
95    988476819    69594919   
104   817473407    61444478   
107   817607712    68398637   
109   817678113    68447779   

                                                                             product_name  \
14                                                 truMedic® InstaShiatsu Pillow Massager   
40                           Human Touch® Reflex SWING Pro Foot, Calf, and Thigh Massager   
82                                           Brookstone® Max 2 Percussion Massager in Red   
91                                                    Sharper Image® BodyScan Seat Topper   
0    HoMedics Cordless SHIATSUTALK™ Voice-Controlled Neck and Shoulder Massager with Heat   
15                           HoMedics Therapist Select 2.0 Foot & Calf Massager with Heat   
16                       HoMedics® Cordless True Touch Neck & Shoulder Massager with Heat   
17                                      HoMedics® 3D True-Touch Massage Cushion With Heat   
23                        HoMedics® Shiatsu Air Max Deep Kneading Foot Massager with Heat   
28                                          HoMedics® Air Compression Back-Stretching Mat   
29                                  HoMedics® Shower Bliss Foot Spa with Heat Boost Power   
34                                                      Wagan Heated Shiatsu Massage Belt   
36          HoMedics® 2-in-1 Shiatsu Massaging Seat Topper with Removeable Massage Pillow   
43                                             Westinghouse Multifunctional Foot Massager   
47        Aurora Health and Beauty Neck and Back Shiatsu Massager with Heat in Tan/Orange   
49                          Brookstone® 2-in-1 Tapping & Shiatsu Neck & Shoulder Massager   
51                                      HoMedics® Shiatsu Air Pro Foot Massager with Heat   
52            Aurora Health and Beauty® Compression Shiatsu Heated Foot Massager in White   
57                              HoMedics® Cordless Percussion Massager with Heat in White   
61                                                        Shiatsu Foot Massager with Heat   
62                             HoMedics® Shiatsu Recline Massaging Chaise Lounger in Grey   
66                           HoMedics® Gentle Touch Gel Shiatsu Massage Cushion with Heat   
69                                         Prospera® TENS Socks Electronic Pulse Massager   
74                                                   Wagan Heated Shiatsu Massage Cushion   
77                                       HoMedics® Gentle Touch Gel Foot Massager in Grey   
78                                                    Prospera Neck and Shoulder Massager   
80                                      HoMedics® Body Flex Air Compression Mat with Heat   
83                 Aurora Health and Beauty® Shiatsu Massage Heated Seat Cushion in Black   
87                                            Brookstone® C7 Shiatsu Massging Seat Topper   
89                                             HoMedics® Shiatsu Massage Pillow with Heat   
94   

In [12]:
error

NameError: name 'error' is not defined

In [ ]:
x=pd.DataFrame()
x['col']=[1.00, 1.10, 1.02, 1.003]
x['col']=x['col'].astype(str).apply(lambda quote: quote.replace('.0', ''))
x

In [13]:
matches[curation_col] = matches['matches'].astype(str)+' in'
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

product_id external_id  \
14    988683304    67178070   
40    817391428    69003943   
82    817375528    68398590   
91    988476164    69592377   
0     817476713    62598620   
15    988650641    65696040   
16    817567391    65695968   
17    817567386    65695913   
23    817381182    68715922   
28    817567389    65695951   
29    817186269    62090162   
34    817182980    61322714   
36    988481024    69604528   
43   1027808045    69639437   
47    817180304    60664969   
49    817607713    68398552   
51    817567387    65696026   
52    817306484    64997124   
57    817318619    65696019   
61    817607710    68398620   
62    817567388    65695944   
66    988480832    69604349   
69   1170775633    66857426   
74    817182979    61322707   
77    988476912    69595006   
78    817183712    61444508   
80    988476820    69594917   
83   1139460437    64997117   
87    817678107    68447724   
89    817318621    65695975   
94    988476887    69594997   
95    988476819    69594919   
104   817473407    61444478   
107   817607712    68398637   
109   817678113    68447779   

                                                                             product_name  \
14                                                 truMedic® InstaShiatsu Pillow Massager   
40                           Human Touch® Reflex SWING Pro Foot, Calf, and Thigh Massager   
82                                           Brookstone® Max 2 Percussion Massager in Red   
91                                                    Sharper Image® BodyScan Seat Topper   
0    HoMedics Cordless SHIATSUTALK™ Voice-Controlled Neck and Shoulder Massager with Heat   
15                           HoMedics Therapist Select 2.0 Foot & Calf Massager with Heat   
16                       HoMedics® Cordless True Touch Neck & Shoulder Massager with Heat   
17                                      HoMedics® 3D True-Touch Massage Cushion With Heat   
23                        HoMedics® Shiatsu Air Max Deep Kneading Foot Massager with Heat   
28                                          HoMedics® Air Compression Back-Stretching Mat   
29                                  HoMedics® Shower Bliss Foot Spa with Heat Boost Power   
34                                                      Wagan Heated Shiatsu Massage Belt   
36          HoMedics® 2-in-1 Shiatsu Massaging Seat Topper with Removeable Massage Pillow   
43                                             Westinghouse Multifunctional Foot Massager   
47        Aurora Health and Beauty Neck and Back Shiatsu Massager with Heat in Tan/Orange   
49                          Brookstone® 2-in-1 Tapping & Shiatsu Neck & Shoulder Massager   
51                                      HoMedics® Shiatsu Air Pro Foot Massager with Heat   
52            Aurora Health and Beauty® Compression Shiatsu Heated Foot Massager in White   
57                              HoMedics® Cordless Percussion Massager with Heat in White   
61                                                        Shiatsu Foot Massager with Heat   
62                             HoMedics® Shiatsu Recline Massaging Chaise Lounger in Grey   
66                           HoMedics® Gentle Touch Gel Shiatsu Massage Cushion with Heat   
69                                         Prospera® TENS Socks Electronic Pulse Massager   
74                                                   Wagan Heated Shiatsu Massage Cushion   
77                                       HoMedics® Gentle Touch Gel Foot Massager in Grey   
78                                                    Prospera Neck and Shoulder Massager   
80                                      HoMedics® Body Flex Air Compression Mat with Heat   
83                 Aurora Health and Beauty® Shiatsu Massage Heated Seat Cushion in Black   
87                                            Brookstone® C7 Shiatsu Massging Seat Topper   
89                                             HoMedics® Shiatsu Massage Pillow with Heat   
94   

In [ ]:
## KWP
# matches[curation_col]=matches['matches']
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [ ]:
error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [14]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [15]:
looks_good('Bed Bath & Beyond', attribute, df, matches)